## Set-Up

In [42]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, inspect

In [43]:
# Create an engine for the `wine_db.sqlite` database
engine = create_engine("sqlite:///Resources/wine_data.db")

In [44]:
inspector = inspect(engine)
inspector.get_table_names()

['winequalityRed', 'winequalityWhite']

In [45]:
red_wine = engine.execute('SELECT * FROM winequalityRed').fetchall()

In [46]:
wine_red_df = pd.DataFrame(red_wine, columns=["wine_id","fixed acidity","volatile acidity", "citric acid",
                                "residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide",
                                "density","pH","sulphates","alcohol","quality"])

In [47]:
# Separate feature variables and target variable
X = wine_red_df.drop(['quality','wine_id'], axis = 1)
y = wine_red_df['quality']

In [48]:
# Normalize feature variables
from sklearn.preprocessing import StandardScaler
X_features = X
scalar = StandardScaler().fit(X)
X = StandardScaler().fit_transform(X)

In [49]:
# Splitting the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=0)

In [50]:
from sklearn.metrics import classification_report
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier(bootstrap=True,random_state=1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.72      0.76       151
           1       0.79      0.86      0.83       193

    accuracy                           0.80       344
   macro avg       0.80      0.79      0.79       344
weighted avg       0.80      0.80      0.79       344



In [51]:
import pickle
pkl_filename = "app/red_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model,file)

scaler = StandardScaler().fit(X)
pkl_scaled_filename = 'app/scaler_red.pkl'
with open(pkl_scaled_filename, 'wb') as file:
    pickle.dump(scaler,file)

In [52]:
# Function to Load and run model with input scores

def red_model(user_input_array):
    import pickle
    #Load Model
    with open('app/red_model.pkl', 'rb') as file:
        red_model = pickle.load(file)
    with open('app/scaler_red.pkl', 'rb') as file:
        scaler = pickle.load(file)

    X = user_input_array.reshape(1,-1)
    X_scaled = scaler.transform(X)

    # Calculate Wine quality
    y_pred_red = red_model.predict(X_scaled)

    return y_pred_red

In [53]:
test = X_test[0]
answer = y[0]
print(test, answer)

[-0.75398448  1.43194831 -0.94397982 -0.39415393  0.11909871 -0.52986225
 -0.87977495 -0.55525786  0.52891085 -1.17113651 -0.39666305] 0


In [54]:
red_model(test)

array([0])

In [57]:
wine_red_df.head()

,wine_id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [60]:
wine_red_df.iloc[3:5,:]

,wine_id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
3,3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
